# Reading File

In [41]:
import pandas as pd

# Initialize a list to store parsed data
data = []

# Read the file
with open('Trade logs//SHMOD4 4L_Trades.txt', 'r') as file:
    lines = file.readlines()
    
    # Loop through each line
    for line in lines:
        if '|' in line:  # Skip lines that do not contain '|'
            # Split the line by the '|' separator
            fields = line.split('|')
            
            # Remove leading and trailing whitespace from each field
            fields = [field.strip() for field in fields]
            # print(fields)
            
            
            # Append the fields to the data list
            data.append(fields)
            # break

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=[
    'Timestamp', 'Description', 'TRDPARITY', 'QTY', 'ParityWas', 
    'OpnCls', 'OrdNumL1', 'OrdNumL2', 'OrdNumL3', 'OrdNumL4', 'M2M'
])

# Define a function to clean the values
def clean_values(series, col_name):
    return series.str.replace(f'{col_name}:', '', regex=False)

# Apply the function to each column except Timestamp and Description
for col in df.columns:
    if col not in ['Timestamp', 'Description']:
        df[col] = clean_values(df[col], col)

# Display the cleaned DataFrame
columns_to_remove = ['OrdNumL1','OrdNumL2','OrdNumL3','OrdNumL4','M2M']
df.drop(columns=columns_to_remove, inplace=True)

# Convert specific columns to integer
df['TRDPARITY'] = df['TRDPARITY'].astype(int)/ 100
# df['TRDPARITY'] = df['TRDPARITY'] / 100

df['QTY'] = df['QTY'].astype(int)
df['OpnCls'] = df['OpnCls'].astype(int)

df['ParityWas'] = df['ParityWas'].astype(int)/ 100
# df['ParityWas'] = df['ParityWas'] / 100

# Rename multiple columns
new_columns = {
    'Description': 'Portfolio',
    'ParityWas': 'ParityAsked',
}

df = df.rename(columns=new_columns)
df = df[df['ParityAsked'] != 5000]
df

,Timestamp,Portfolio,TRDPARITY,QTY,ParityAsked,OpnCls
0,2024-May-21 09:15:04,MIDCPNIFTY24MAY-11325-11300PE,1.50,1,0.85,1
1,2024-May-21 09:15:05,MIDCPNIFTY24MAY-11350-11300PE,-4.40,2,0.90,1
2,2024-May-21 09:15:06,FINNIFTY24521-21450-21500CE,0.20,1,0.55,1
3,2024-May-21 09:15:13,FINNIFTY24521-21800-21600PE,1.20,2,1.10,1
4,2024-May-21 09:15:15,NIFTY24MAY-22100-22500CE,5.60,2,2.50,1
...,...,...,...,...,...,...
1969,2024-May-21 15:29:50,NIFTY24523-22800-22600PE,1.10,1,0.90,1
1970,2024-May-21 15:29:55,NIFTY24523-22450-22500CE,0.65,2,0.85,1
1971,2024-May-21 15:29:57,NIFTY24523-22450-22600CE,1.32,2,0.40,2
1972,2024-May-21 15:29:57,NIFTY24523-22450-22550CE,1.30,2,0.35,2


# Alpha Calculation 

In [42]:
def calculate_alpha(row):
    if 'BANKNIFTY' in row['Portfolio']:
        lot_size = 15
    elif 'FINNIFTY' in row['Portfolio']:
        lot_size = 25
    elif 'MIDCPNIFTY' in row['Portfolio']:
        lot_size = 50
    elif 'NIFTY' in row['Portfolio']:
        lot_size = 25
    
    alpha = (row['TRDPARITY'] - row['ParityAsked']) * row['QTY'] * lot_size
    return alpha

# Apply the function to each row
df['alpha'] = df.apply(calculate_alpha, axis=1)
df

,Timestamp,Portfolio,TRDPARITY,QTY,ParityAsked,OpnCls,alpha
0,2024-May-21 09:15:04,MIDCPNIFTY24MAY-11325-11300PE,1.50,1,0.85,1,9.75
1,2024-May-21 09:15:05,MIDCPNIFTY24MAY-11350-11300PE,-4.40,2,0.90,1,-159.00
2,2024-May-21 09:15:06,FINNIFTY24521-21450-21500CE,0.20,1,0.55,1,-14.00
3,2024-May-21 09:15:13,FINNIFTY24521-21800-21600PE,1.20,2,1.10,1,8.00
4,2024-May-21 09:15:15,NIFTY24MAY-22100-22500CE,5.60,2,2.50,1,155.00
...,...,...,...,...,...,...,...
1969,2024-May-21 15:29:50,NIFTY24523-22800-22600PE,1.10,1,0.90,1,5.00
1970,2024-May-21 15:29:55,NIFTY24523-22450-22500CE,0.65,2,0.85,1,-10.00
1971,2024-May-21 15:29:57,NIFTY24523-22450-22600CE,1.32,2,0.40,2,46.00
1972,2024-May-21 15:29:57,NIFTY24523-22450-22550CE,1.30,2,0.35,2,47.50


In [43]:
positive_alpha = round(df[df['alpha'] > 0]['alpha'].sum(),2)
negative_alpha = round(df[df['alpha'] < 0]['alpha'].sum(),2)
net_alpha = round(positive_alpha + negative_alpha,2)

print(f'% positive_alpha : {positive_alpha}\n% negative_alpha : {negative_alpha}\n% net_alpha : {net_alpha}')

% positive_alpha : 17920.2
% negative_alpha : -14287.95
% net_alpha : 3632.25


In [44]:
positive_alpha_events = f'{round(((df['alpha'] >= 0).sum()/len(df))*100,2)} %'
negative_alpha_events = f'{round(((df['alpha'] < 0).sum()/len(df))*100,2)} %'

print(f'% positive_alpha_events : {positive_alpha_events}\n% negative_alpha_events : {negative_alpha_events}')

% positive_alpha_events : 71.36 %
% negative_alpha_events : 28.64 %


# Top 5 Gainers/Loosers

In [45]:
# Group by Portfolio and calculate open_qty, close_qty, buy_margin, and sell_margin

df['price'] = df['TRDPARITY'] * df['QTY']

# Determine lot_size
def determine_lot_size(portfolio):
    if 'BANKNIFTY' in portfolio:
        return 15
    elif 'FINNIFTY' in portfolio:
        return 25
    elif 'MIDCPNIFTY' in portfolio:
        return 50
    elif 'NIFTY' in portfolio:
        return 25
    return 1

result = df.groupby('Portfolio').agg(
    open_qty=pd.NamedAgg(column='QTY', aggfunc=lambda x: x[df.loc[x.index, 'OpnCls'] == 1].sum()),
    close_qty=pd.NamedAgg(column='QTY', aggfunc=lambda x: x[df.loc[x.index, 'OpnCls'] == 2].sum()),
    buy_margin=pd.NamedAgg(column='price', aggfunc=lambda x: x[df.loc[x.index, 'OpnCls'] == 1].sum()),
    sell_margin=pd.NamedAgg(column='price', aggfunc=lambda x: x[df.loc[x.index, 'OpnCls'] == 2].sum())
).reset_index()

# Calculate buy_avg and sell_avg
result['buy_avg'] = result.apply(lambda row: row['buy_margin'] / row['open_qty'] if row['open_qty'] != 0 else 0, axis=1)
result['sell_avg'] = result.apply(lambda row: row['sell_margin'] / row['close_qty'] if row['close_qty'] != 0 else 0, axis=1)

# Calculate gross_profit
result['lot_size'] = result['Portfolio'].apply(determine_lot_size)
result['gross_profit'] = ((result['open_qty'] * result['buy_avg']) + (result['close_qty'] * result['sell_avg'])) * result['lot_size']

def extract_strike_diff(portfolio):
    # Split the string based on the hyphens
    parts = portfolio.split('-')
    
    # The last two parts should be the strike prices
    if len(parts) >= 3:
        strike1 = int(parts[-2])
        strike2 = int(parts[-1][:-2])  # Removing the last two characters (e.g., 'CE' or 'PE')
        
        # Calculate and return the difference
        return abs(strike1 - strike2)
    else:
        raise ValueError("Invalid portfolio format")

result['diff_btw_strikes'] = result['Portfolio'].apply(extract_strike_diff)

# Calculate sum_of_qty
result['sum_of_qty'] = result['open_qty'] + result['close_qty']

result['lot_size'] = result['Portfolio'].apply(determine_lot_size)

# Calculate expense
result['expense'] = result.apply(lambda row: row['diff_btw_strikes'] * row['sum_of_qty'] * row['lot_size'] * 0.00105, axis=1)

result['net_profit'] = result['gross_profit'] - result['expense']

# Display the final result
columns_to_remove = ['buy_margin','sell_margin','diff_btw_strikes','sum_of_qty','lot_size','lot_size']
result.drop(columns=columns_to_remove, inplace=True)

result

                       Portfolio  open_qty  close_qty  buy_margin  \
0    FINNIFTY24521-21000-21400CE         1          2        1.80   
1    FINNIFTY24521-21000-21500CE         2          2       -0.25   
2    FINNIFTY24521-21000-21600CE         2          0        1.00   
3    FINNIFTY24521-21050-21300CE         1          1       -2.20   
4    FINNIFTY24521-21050-21400CE         2          2       -1.05   
..                           ...       ...        ...         ...   
208     NIFTY24MAY-23600-22500PE         1          0        6.70   
209     NIFTY24MAY-24000-22450PE        14          0      122.35   
210     NIFTY24MAY-24000-22500PE         7          0       59.60   
211     NIFTY24MAY-24000-22550PE        22          0      185.35   
212     NIFTY24MAY-24500-22650PE         1          0       11.10   

     sell_margin    buy_avg  sell_avg  
0           1.85   1.800000     0.925  
1           2.40  -0.125000     1.200  
2           0.00   0.500000     0.000  
3          

,Portfolio,open_qty,close_qty,buy_avg,sell_avg,gross_profit,expense,net_profit
0,FINNIFTY24521-21000-21400CE,1,2,1.800000,0.925,146.00,50.4000,95.6000
1,FINNIFTY24521-21000-21500CE,2,2,-0.125000,1.200,86.00,84.0000,2.0000
2,FINNIFTY24521-21000-21600CE,2,0,0.500000,0.000,40.00,50.4000,-10.4000
3,FINNIFTY24521-21050-21300CE,1,1,-2.200000,-0.950,-126.00,21.0000,-147.0000
4,FINNIFTY24521-21050-21400CE,2,2,-0.525000,1.250,58.00,58.8000,-0.8000
...,...,...,...,...,...,...,...,...
208,NIFTY24MAY-23600-22500PE,1,0,6.700000,0.000,167.50,28.8750,138.6250
209,NIFTY24MAY-24000-22450PE,14,0,8.739286,0.000,3058.75,569.6250,2489.1250
210,NIFTY24MAY-24000-22500PE,7,0,8.514286,0.000,1490.00,275.6250,1214.3750
211,NIFTY24MAY-24000-22550PE,22,0,8.425000,0.000,4633.75,837.3750,3796.3750


# Final Df

In [46]:
#! Final dataframe 

top_5_df = result.nlargest(5, 'net_profit')
bottom_5_df = result.nsmallest(5, 'net_profit')

top_alpha_portfolio = top_5_df['Portfolio'].to_list()
bottom_alpha_portfolio = bottom_5_df['Portfolio'].to_list()

data = {
        'dealer_id' : [98765],

        'positive_alpha_events' : [positive_alpha_events],
        'negative_alpha_events' : [negative_alpha_events],
        
        'positive_alpha' : [positive_alpha],
        'negative_alpha' : [negative_alpha],
        'net_alpha' : [net_alpha],

        't1' : [top_alpha_portfolio[0]],
        't2' : [top_alpha_portfolio[1]],
        't3' : [top_alpha_portfolio[2]],
        't4' : [top_alpha_portfolio[3]],
        't5' : [top_alpha_portfolio[4]],

        'b1' : [bottom_alpha_portfolio[0]],
        'b2' : [bottom_alpha_portfolio[1]],
        'b3' : [bottom_alpha_portfolio[2]],
        'b4' : [bottom_alpha_portfolio[3]],
        'b5' : [bottom_alpha_portfolio[4]]   
    }

df = pd.DataFrame(data)
df

,dealer_id,positive_alpha_events,negative_alpha_events,positive_alpha,negative_alpha,net_alpha,t1,t2,t3,t4,t5,b1,b2,b3,b4,b5
0,98765,71.36 %,28.64 %,17920.2,-14287.95,3632.25,FINNIFTY24MAY-21100-21500CE,NIFTY24MAY-22100-22600CE,NIFTY24MAY-24000-22550PE,NIFTY24MAY-24000-22450PE,FINNIFTY24521-21450-21400PE,MIDCPNIFTY24MAY-11350-11300PE,FINNIFTY24521-21750-21300PE,FINNIFTY24521-21800-21500PE,FINNIFTY24521-21150-21400CE,FINNIFTY24521-21600-21300PE


# Rough